In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import bs4
from urllib.request import urlopen as uReq
from urllib.request import Request, urlopen
from unidecode import unidecode

import os
import re
import pickle
import json

In [2]:
# tapology_fighters = json.load(open('fighters_25-05-19.txt','r'))
# ufc_stats_fighters = json.load(open('all_ufc_stats_info_May_22','r'))
# sherdog_fighters = json.load(open('fighters_sherdog_May_22','r'))
# site_rankings = json.load(open('ranking_dict','r'))

In [3]:
tapology_fighters = json.load(open('tapology_fighters_June_3.txt','r'))
ufc_stats_fighters = json.load(open('all_ufc_stats_info_June_3','r'))
sherdog_fighters = json.load(open('fighters_sherdog_June_3','r'))

In [4]:
site_rankings = json.load(open('ranking_dict','r'))

In [5]:
len(tapology_fighters),len(ufc_stats_fighters),len(sherdog_fighters)

(1934, 1934, 1934)

In [6]:
tapology_to_ufcstats = {'Alberto Cerra Leon':'Alberta Cerra Leon',
 'Aleksandra Albu':'Alexandra Albu',
 'Alex Ricci':'Alessandro Ricci',
 'Alexey Kunchenko':'Aleksei Kunchenko',
 'Alexey Oleinik':'Aleksei Oleinik',
 'Allen Berubie':'Allen Berube',
 'Alp Ozkilic':'Alptekin Ozkilic',
 'Alex da Silva':'Alex Da Silva',
 'Alexander Torres':'Alex Torres',
 'Anderson dos Santos':'Anderson Dos Santos',
 'Andrey Semenov':'Andrei Semenov',
 'Antonio dos Santos Jr.':'Antonio Dos Santos',
 'Anying Wang':'An Ying Wang',
 'Bharat Khandare':'Bharat Kandare',
 'Brad Scott':'Bradley Scott',
 'Bruno Mesquita':'Bruno Korea',
 'C.B. Dollaway':'CB Dollaway',
 'Carls John de Tomas':'Carls John De Tomas',
 'Chan Mi Jeon':'Chanmi Jeon',
 'Chris De La Rocha':'Chris de la Rocha',
 'Cristophe Leninger':'Christophe Leninger',
 'Cristiane Justino':'Cris Cyborg',
 'Dan Spohn':'Daniel Spohn',
 'Daniel Barrera':'Dan Barrera',
 'Daniel Hooker':'Dan Hooker',
 'Dave Kaplan':'David Kaplan',
 'Des Green':'Desmond Green',
 'Dmitrii Smolyakov':'Dmitrii Smoliakov',
 'Dmitriy Sosnovskiy':'Dmitry Sosnovskiy',
 'Dmitriy Stepanov':'Dmitri Stepanov',
 'Doo Ho Choi':'Dooho Choi',
 'Douglas Evans':'Doug Evans',
 'Edwin Dewees':'Edwin DeWees',
 'Eldo Dias Xavier':'Eldo Xavier Dias',
 'Elizeu Zaleski':'Elizeu Zaleski dos Santos',
 'Emil Weber Meek':'Emil Meek',
 'Emily  Kagan':'Emily Peters Kagan',
 'Felix Mitchell':'Felix Lee Mitchell',
 "Francis N'Gannou":'Francis Ngannou',
 'Francisco Rivera Jr.':'Francisco Rivera',
 'Georges St. Pierre':'Georges St-Pierre',
 'Heather Clark':'Heather Jo Clark',
 'Guilherme Bomba':'Guilherme Vasconcelos',
 'Jim Wallhead':'Jimmy Wallhead',
 'Humberto  Brown Morrison':'Humberto Brown',
 'Joao Marcos Pierini':'Joao Pierini',
 'Joey Merritt':'Joe Merritt',
 'John Campatella':'John Campetella',
 'John Macapa':'John Teixeira',
 'John Olav Einemo':'Jon Olav Einemo',
 'Jose Maria Tome':'Jose Maria',
 'Josh Burkman':'Joshua Burkman',
 'Josh Sampo':'Joshua Sampo',
 'Jose Alberto Quinones':'Jose Quinonez',
 'Josh Schockman':'Josh Shockman',
 'Juan Manuel Puig':'Juan Puig',
 'Junior dos Santos':'Junior Dos Santos',
 'Khalil Rountree':'Khalil Rountree Jr.',
 'Kris Rotharmel':'Kristian Rothaermel',
 'LaVerne Clark':'Laverne Clark',
 'Lance Gibson Sr.':'Lance Gibson',
 'Livia Renata Souza':'Livinha Souza',
 'Luiz Dutra Jr.':'Luiz Dutra',
 'Maia Kahaunaele-Stevenson':'Maia Stevenson',
 'Manny Gamburyan':'Manvel Gamburyan',
 'Marcio Alexandre Jr.':'Marcio Alexandre Junior',
 'Kimbo Slice':'Kevin Ferguson',
 'Marco Antonio Beltran':'Marco Beltran',
 'Matt Riddle':'Matthew Riddle',
 'Mike Patt':'Michael Patt',
 'Mike Trizano':'Michael Trizano',
 'Mike Graves':'Michael Graves',
 'Mike De La Torre':'Mike de la Torre',
 'Mirko Filipovic':'Mirko Cro Cop',
 'Antonio Rodrigo Nogueira':'Minotauro Nogueira',
 'Mostapha Al Turk':'Mostapha Al-Turk',
 'Nah-shon Burrell':'Nah-Shon Burrell',
 'Nick Negumereanu':'Nicolae Negumereanu',
 'Orlando Weit':'Orlando Wiet',
 'Ovince St. Preux':'Ovince Saint Preux',
 'Pat Walsh':'Patrick Walsh',
 'Phil De Fries':'Philip De Fries',
 'Quinton Jackson':'Rampage Jackson',
 'Rafael dos Anjos':'Rafael Dos Anjos',
 'Rich Crunkilton':'Richard Crunkilton',
 'Rafael Cavalcante':'Rafael Feijao',
 'Robbie Peralta':'Robert Peralta',
 'Antonio Rogerio Nogueira':'Rogerio Nogueira',
 'Ron Van Clief':'Ron van Clief',
 'Rodolfo Rubio Perez':'Rodolfo Rubio',
 'Sam Morgan':'Sammy Morgan',
 'Ronaldo Souza':'Jacare Souza',
 'Sergey Pavlovich':'Sergei Pavlovich',
 'Sergey Spivak':'Serghei Spivac',
 'Seung Woo Choi':'Seungwoo Choi',
 'Shane Del Rosario':'Shane del Rosario',
 'Steven Kennedy':'Steve Kennedy',
 'Steven Lynch':'Stevie Lynch',
 'T.J. Dillashaw':'TJ Dillashaw',
 "T.J. O'Brien":"TJ O'Brien",
 'Thomas Speer':'Tommy Speer',
 'Timothy Johnson':'Tim Johnson',
 'Tsuyoshi Kosaka':'Tsuyoshi Kohsaka',
 'William Macario':'William Patolino',
 'Xiaonan Yan':'Yan Xiaonan',
 'Yadong Song':'Song Yadong',
 'Yanan Wu':'Wu Yanan',
 'Yaozong Hu':'Hu Yaozong',
 'Seo Hee Ham':'Seohee Ham',
 'Tony Christodoulou':'Anthony Christodoulou',
 'Douglas Andrade':'Douglas Silva de Andrade',
 'Luis Henrique da Silva':'Henrique da Silva',
 'Jian Ping Yang':'Jianping Yang',
 'Edimilson Souza':'Kevin Souza',
 'Guan Wang':'Wang Guan',
 'Lipeng Zhang':'Zhang Lipeng',
 'Azunna Anyanwu':'Zu Anyanwu',
 'Zhikui Yao':'Yao Zhikui',
 'War Machine':'Jon Koppenhaver',
 'Kenan Song':'Song Kenan',
 'Ronald Stallings':'Ron Stallings',
 'Roli Delgado':'Rolando Delgado',
 'Robert Whiteford':'Rob Whiteford',
 'Guangyou Ning':'Ning Guangyou',
 'Mike Van Arsdale':'Mike van Arsdale',
 'Ramiro Hernandez':'Junior Hernandez',
 'Leonardo Guimaraes':'Leonardo Augusto Leleco',
 'Jingliang Li':'Li Jingliang',
 'Pingyuan Liu':'Liu Pingyuan',
 'Tuerxun Jumabieke':'Jumabieke Tuerxun',
 'Kazuo Takahashi':'Yoshiki Takahashi'}
ufcstats_to_tapology = {v: k for k, v in tapology_to_ufcstats.items()}

In [7]:
sherdog_to_ufcstats = {'Alan Patrick Silva Alves':'Alan Patrick',
 'Alberto Cerra Leon':'Alberta Cerra Leon',
 'Alexey Oleynik':'Aleksei Oleinik',
 'Alex da Silva Coelho':'Alex Da Silva',
 'Amanda Bobby Cooper':'Amanda Cooper',
 'Aleksandra Albu':'Alexandra Albu',
 'Anderson dos Santos':'Anderson Dos Santos',
 'Alexey Kunchenko':'Aleksei Kunchenko',
 'Antonio Carlos Jr.':'Antonio Carlos Junior',
 'Antonio dos Santos Jr.':'Antonio Dos Santos',
 'Anying Wang':'An Ying Wang',
 'B.J. Penn':'BJ Penn',
 'Arjan Singh Bhullar':'Arjan Bhullar',
 'Anistavio Medeiros de Figueiredo':'Anistavio Medeiros',
 'Alberto Emiliano Pereira':'Alberto Uda',
 'Ben Alloway':'Benny Alloway',
 'Bharat Khandare':'Bharat Kandare',
 'C.B. Dollaway':'CB Dollaway',
 'Bruno Rodrigues Mesquita':'Bruno Korea',
 'C.J. Keith':'CJ Keith',
 'Carlo Pedersoli Jr.':'Carlo Pedersoli',
 'Carls John de Tomas':'Carls John De Tomas',
 'Cesar Marcussi':'Cesar Marscucci',
 'Claudio Henrique da Silva':'Claudio Silva',
 'Dan Spohn':'Daniel Spohn',
 'Daniel Barrera':'Dan Barrera',
 'Daniel Downes':'Danny Downes',
 'Daniel Stittgen':'Dan Stittgen',
 'Dave Roberts':'David Roberts',
 'Darrell Gholar':'Darrel Gholar',
 'Carlos Diego Ferreira':'Diego Ferreira',
 'Dieuseul Berto':'Dieusel Berto',
 'Dmitriy Sosnovskiy':'Dmitry Sosnovskiy',
 'Dmitry Smoliakov':'Dmitrii Smoliakov',
 'Doo Ho Choi':'Dooho Choi',
 'Douglas Evans':'Doug Evans',
 'Duda Santanna':'Duda Santana',
 'Edwin Dewees':'Edwin DeWees',
 'Emily Kagan':'Emily Peters Kagan',
 'Eldo Dias Xavier':'Eldo Xavier Dias',
 'Felipe Dias Colares':'Felipe Colares',
 'Georges St. Pierre':'Georges St-Pierre',
 'Felix Mitchell':'Felix Lee Mitchell',
 'Enrique Briones':'Henry Briones',
 'Godofredo Castro':'Godofredo Pepey',
 'Geraldo de Freitas Jr.':'Geraldo de Freitas',
 'He-Man Ali Gipson':'He-Man Gipson',
 'Henrique     da Silva':'Henrique da Silva',
 'J.C. Cottrell':'JC Cottrell',
 'J.J. Aldrich':'JJ Aldrich',
 'Jack McLaughlin':'Jack McGlaughlin',
 'Jim Crute':'Jimmy Crute',
 'Jim Hettes':'Jimy Hettes',
 'Jim Wallhead':'Jimmy Wallhead',
 'Joao Marcos Pierini':'Joao Pierini',
 'John Olav Einemo':'Jon Olav Einemo',
 'Jose Alberto Quinonez':'Jose Quinonez',
 'Jose Maria Tome':'Jose Maria',
 'John Teixeira da Conceicao':'John Teixeira',
 'Jon delos Reyes':'Jon Delos Reyes',
 'Jorge Antonio Cezario de Oliveira':'Jorge de Oliveira',
 'Josh Sampo':'Joshua Sampo',
 'Josh Schockman':'Josh Shockman',
 'Joseph Duffy':'Joe Duffy',
 'Juan Francisco Espino Dieppa':'Juan Espino',
 'Juan Manuel Puig':'Juan Puig',
 'Junie Allen Browning':'Junie Browning',
 'Junior dos Santos':'Junior Dos Santos',
 'K.J. Noons':'KJ Noons',
 'Khalil Rountree':'Khalil Rountree Jr.',
 'LaVerne Clark':'Laverne Clark',
 'Leonardo Mafra Texeira':'Leonardo Mafra',
 'Luis Henrique Barbosa de Oliveira':'Luis Henrique',
 'Luiz Jorge Dutra Jr.':'Luiz Dutra',
 'Maia Kahaunaele-Stevenson':'Maia Stevenson',
 'Manny Gamburyan':'Manvel Gamburyan',
 'Marcio Alexandre Jr.':'Marcio Alexandre Junior',
 'Marcos Vinicius Borges Pancini':'Marcos Vinicius',
 'Matheus Nicolau Pereira':'Matheus Nicolau',
 'Matt Riddle':'Matthew Riddle',
 'Michael Guymon':'Mike Guymon',
 'Mike Thomas Brown':'Mike Brown',
 'Mike Van Arsdale':'Mike van Arsdale',
 'Mirko Filipovic':'Mirko Cro Cop',
 'Antonio Rogerio Nogueira':'Rogerio Nogueira',
 'Antonio Rodrigo Nogueira':'Minotauro Nogueira',
 'Nicholas Musoke':'Nico Musoke',
 'Ovince St. Preux':'Ovince Saint Preux',
 'Paulo Henrique Costa':'Paulo Costa',
 'Rafael dos Anjos':'Rafael Dos Anjos',
 'Robert Whiteford':'Rob Whiteford',
 'Rich Crunkilton':'Richard Crunkilton',
 'Robert Emerson':'Rob Emerson',
 'Renato Carneiro':'Renato Moicano',
 'Robbie Peralta':'Robert Peralta',
 'Ronaldo Souza':'Jacare Souza',
 'Rodrigo Goiana de Lima':'Rodrigo de Lima',
 'Roli Delgado':'Rolando Delgado',
 'Rony Mariano Bezerra':'Rony Jason',
 'Sako Chivitchyan':'Sako Chivitchian',
 'Sam Morgan':'Sammy Morgan',
 'Seo Hee Ham':'Seohee Ham',
 'Sergey Spivak':'Serghei Spivac',
 'Steven Lynch':'Stevie Lynch',
 'Steven Ray':'Stevie Ray',
 'Sung Woo Choi':'Seungwoo Choi',
 'T.J. Dillashaw':'TJ Dillashaw',
 'T.J. Grant':'TJ Grant',
 "T.J. O'Brien":"TJ O'Brien",
 'T.J. Waldburger':'TJ Waldburger',
 'Thiago de Oliveira Perpetuo':'Thiago Perpetuo',
 'Tiago dos Santos e Silva':'Tiago Trator',
 'Timothy Johnson':'Tim Johnson',
 'Tom Speer':'Tommy Speer',
 'Tsuyoshi Kosaka':'Tsuyoshi Kohsaka',
 'Wendell de Oliveira Marques':'Wendell Oliveira',
 'Xiaonan Yan':'Yan Xiaonan',
 'Yadong Song':'Song Yadong',
 'Yanan Wu':'Wu Yanan',
 'Saparbek Safarov':'Saparbeg Safarov',
 'Vinicius Kappke de Queiroz':'Vinicius Queiroz',
 'Yuta Sasaki':'Ulka Sasaki',
 'Quinton Jackson':'Rampage Jackson',
 'Phil Brooks':'CM Punk',
 'Cristiane Justino':'Cris Cyborg',
 'Leonardo Guimaraes':'Leonardo Augusto Leleco',
 'Jussier da Silva':'Jussier Formiga',
 'Carlos Augusto Inocente Filho':'Guto Inocente',
 'Guan Wang':'Wang Guan',
 'Patrick Holohan':'Paddy Holohan',
 'Kenan Song':'Song Kenan',
 'Azunna Anyanwu':'Zu Anyanwu',
 'Kazuo Takahashi':'Yoshiki Takahashi',
 'William Macario':'William Patolino',
 'Marco Polo Reyes':'Polo Reyes',
 'Wagner Silva Gomes':'Wagner Silva',
 'Edimilson Souza':'Kevin Souza',
 'Pingyuan Liu':'Liu Pingyuan',
 'Rafael Cavalcante':'Rafael Feijao',
 'Ramiro Hernandez':'Junior Hernandez',
 'Yaozong Hu':'Hu Yaozong',
 'Heather Clark':'Heather Jo Clark',
 'Jingliang Li':'Li Jingliang',
 'Guangyou Ning':'Ning Guangyou',
 'Zhikui Yao':'Yao Zhikui',
 'Lipeng Zhang':'Zhang Lipeng',
 'War Machine':'Jon Koppenhaver',
 'Benjamin Duffy':'James Giboo'}

ufcstats_to_sherdog = {v: k for k, v in sherdog_to_ufcstats.items()}

In [8]:
set_ufc = set([x['Name'] for x in ufc_stats_fighters])
set_tap = set([x['Name'] for x in tapology_fighters])
set_sher = set([x['Name'] for x in sherdog_fighters])

In [9]:
for name in (set_sher - set_ufc):
    if name not in sherdog_to_ufcstats:
        print('In sherdog there is no name from ufc_stats',name)

In [10]:
for name in (set_tap - set_ufc):
    if name not in tapology_to_ufcstats:
        print('In tapology there is no name from ufc_stats',name)

In [11]:
rankings_to_ufc_stats = {'Jussier Da Silva':'Jussier Formiga',
 'Constantinos Philippou':'Costas Philippou',
 'Antonio Rogerio Nogueira':'Rogerio Nogueira',
 'Antonio Rodrigo Nogueira':'Minotauro Nogueira',
 'Ronaldo Souza':'Jacare Souza',
 'T.J. Dillashaw':'TJ Dillashaw',
 'Rafael Cavalcante':'Rafael Feijao',
 'Bigfoot Silva':'Antonio Silva',
 'Heather Clark':"Heather Jo Clark",
 'Jessica Rose Clark':"Jessica-Rose Clark",
 'Elizeu dos Santos':"'Elizeu Zaleski dos Santos'",
 'Kai Kara France':"Kai Kara-France",
 'Casey  Kenney':'Casey Kenney'}
ufcstats_to_rankings = {v: k for k, v in rankings_to_ufc_stats.items()}

In [12]:
combined = []
for ufc_stat_fgtr in ufc_stats_fighters:
    name = ufc_stat_fgtr['Name']
    batch  = [ufc_stat_fgtr]

    if name in ufcstats_to_sherdog:
        sherdog_name = ufcstats_to_sherdog[name]
    else: sherdog_name = name
    batch += [x for x in sherdog_fighters if x['Name'] == sherdog_name]

    if name in ufcstats_to_tapology:
        tapology_name = ufcstats_to_tapology[name]
    else: tapology_name = name
    batch += [x for x in tapology_fighters if x['Name'] == tapology_name]

    if name in ufcstats_to_rankings:
        rankings_name = ufcstats_to_rankings[name]
    else: rankings_name = name
    
    if rankings_name in site_rankings:
        batch.append(site_rankings[rankings_name])
    else:
        batch.append({})
    
    combined.append(batch)

In [13]:
json.dump(combined,open('combined_dictionary_June_3','w'))